# `pandas` VS SQL Report

## Abstract
`pandas` is a powerful Python data analysis library that provides a wide range of functions for manipulating, filtering, and transforming datasets. When used effectively, many of its capabilities can replicate and often extend the functionality of SQL. This report demonstrates how to use `pandas` to reproduce common SQL queries, showing practical, side-by-side equivalents. Through these examples, readers will gain a solid understanding of how `pandas` can be applied to perform SQL-like operations in a Python environment.

## Demonstration
### Prepare Database
In this section, we define the database path that will store the data from the `.csv` files. By using a database, we can relate and combine data from different files more easily.

1. Create database file and connection.

In [32]:
import sqlite3

db_path = 'data/NYC-Flights-Data/nycflights13.db'
conn = sqlite3.connect(db_path)

2. Read CSVs with `pandas` & export to SQLite

In [33]:
import pandas as pd

# Read gzipped CSV files
flights = pd.read_csv('data/NYC-Flights-Data/nycflights13_flights.csv.gz', compression="gzip", comment="#")
airlines = pd.read_csv('data/NYC-Flights-Data/nycflights13_airlines.csv.gz', compression="gzip", comment="#")
airports = pd.read_csv('data/NYC-Flights-Data/nycflights13_airports.csv.gz', compression="gzip", comment="#")
planes = pd.read_csv('data/NYC-Flights-Data/nycflights13_planes.csv.gz', compression="gzip", comment="#")
weather = pd.read_csv('data/NYC-Flights-Data/nycflights13_weather.csv.gz', compression="gzip", comment="#")

# Write each DataFrame to table in SQLite
flights.to_sql('flights', conn, if_exists='replace', index=False)
airlines.to_sql('airlines', conn, if_exists='replace', index=False)
airports.to_sql('airports', conn, if_exists='replace', index=False)
planes.to_sql('planes', conn, if_exists='replace', index=False)
weather.to_sql('weather', conn, if_exists='replace', index=False)

26130

3. Check data.

In [34]:
print(flights.head(2))
print(airlines.head(2))
print(airports.head(2))
print(planes.head(2))
print(weather.head(2))

   year  month  day  dep_time  sched_dep_time  dep_delay  arr_time  \
0  2013      1    1     517.0             515        2.0     830.0   
1  2013      1    1     533.0             529        4.0     850.0   

   sched_arr_time  arr_delay carrier  flight tailnum origin dest  air_time  \
0             819       11.0      UA    1545  N14228    EWR  IAH     227.0   
1             830       20.0      UA    1714  N24211    LGA  IAH     227.0   

   distance  hour  minute            time_hour  
0      1400     5      15  2013-01-01 05:00:00  
1      1416     5      29  2013-01-01 05:00:00  
  carrier                    name
0      9E       Endeavor Air Inc.
1      AA  American Airlines Inc.
   faa                           name        lat        lon   alt  tz dst  \
0  04G              Lansdowne Airport  41.130472 -80.619583  1044  -5   A   
1  06A  Moton Field Municipal Airport  32.460572 -85.680028   264  -6   A   

              tzone  
0  America/New_York  
1   America/Chicago  
  tailn

### `panda` Example Application

In this section, we demonstrate the usage of `pandas` by writing code that produces results equivalent to those obtained using SQL.

```sql
SELECT DISTINCT engine from planes
```

In [35]:
# Describe the plane table
t_planes = pd.read_sql_query("PRAGMA table_info(planes);", conn)
print(t_planes)

   cid          name     type  notnull dflt_value  pk
0    0       tailnum     TEXT        0       None   0
1    1          year     REAL        0       None   0
2    2          type     TEXT        0       None   0
3    3  manufacturer     TEXT        0       None   0
4    4         model     TEXT        0       None   0
5    5       engines  INTEGER        0       None   0
6    6         seats  INTEGER        0       None   0
7    7         speed     REAL        0       None   0
8    8        engine     TEXT        0       None   0


In [36]:
from pandas.testing import assert_frame_equal

task1_sql = pd.read_sql_query("SELECT DISTINCT engine FROM planes ORDER BY engine;", conn)
task1_my = planes[["engine"]].drop_duplicates().sort_values("engine").reset_index(drop=True)

assert_frame_equal(task1_sql, task1_my)

The query selects the column `engine` from the `planes` dataset and removes duplicate combinations.

```sql
SELECT DISTINCT type, engine FROM planes
```

In [37]:
task2_sql = pd.read_sql_query("SELECT DISTINCT type, engine FROM planes ORDER BY type,engine", conn)

task2_my = planes[["type", "engine"]].drop_duplicates().sort_values(["type", "engine"]).reset_index(drop=True)

assert_frame_equal(task2_sql, task2_my)

The query selects the columns `type` and `engine` from the `planes` dataset and removes duplicate combinations.

```sql
SELECT COUNT(*), engine FROM planes GROUP BY engine
```

In [38]:
task3_sql = pd.read_sql_query("SELECT COUNT(*) as count, engine FROM planes GROUP BY engine", conn)

task3_my = planes.groupby(["engine"]).size().reset_index(name="count")
task3_my = task3_my[["count", "engine"]]

assert_frame_equal(task3_sql, task3_my)

This query groups all rows in the `planes` table by the value of the `engine` column, counts how many planes fall into each engine category, and returns a table that lists each engine type along with the number of planes using the engine.

```sql
SELECT COUNT(*), engine, type FROM planes GROUP BY engine, type
```

In [39]:
task4_sql = pd.read_sql_query("SELECT COUNT(*) as count, engine, type FROM planes GROUP BY engine, type", conn)

task4_my = planes.groupby(["engine", "type"]).size().reset_index(name="count")
task4_my = task4_my[["count", "engine", "type"]]

assert_frame_equal(task4_sql, task4_my)

This query groups all planes by both `engine` and `type`. For each `(engine,type)` combination, it counts how many planes match that combination.

```sql
SELECT MIN(year), AVG(year), MAX(year), engine, manufacturer FROM planes
```

In [40]:
task5_sql = pd.read_sql_query("""
SELECT MIN(year), AVG(year), MAX(year), engine, manufacturer FROM planes GROUP BY engine, manufacturer
""", conn)

task5_my = planes.groupby(["engine", "manufacturer"]).agg({"year": ["min", "mean", "max"]})
# Flatten MultiIndex columns
task5_my.columns = ["MIN(year)", "AVG(year)", "MAX(year)"]
# Move engine + manufacturer into columns and reorder
task5_my = task5_my.reset_index()[["MIN(year)", "AVG(year)", "MAX(year)", "engine", "manufacturer"]]

assert_frame_equal(task5_sql, task5_my)

This query groups all rows in the planes table by both `engine` and `manufacturer`.
For each group, it computes: the minimum production year, the average production year, the maximum production year.
The result contains one row per `(engine, manufacturer)` combination, along with those three statistics.

```sql
SELECT * FROM planes WHERE speed IS NOT NULL
```

In [41]:
task6_sql = pd.read_sql_query("SELECT * FROM planes WHERE speed IS NOT NULL ORDER BY speed", conn)

task6_my = planes[planes["speed"].notnull()].sort_values("speed").reset_index(drop=True)

assert_frame_equal(task6_sql, task6_my)

This query selects all columns from the planes table but only keeps the rows where the speed column is not null. In other words, it filters out all planes with missing speed information.

```sql
SELECT tailnum FROM planes
WHERE seats BETWEEN 150 AND 210 AND year >= 2011
```

In [42]:
task7_sql = pd.read_sql_query("""
SELECT tailnum FROM planes WHERE seats BETWEEN 150 AND 210 AND year >= 2011
""", conn)
task7_sql = task7_sql.sort_values("tailnum").reset_index(drop=True)

task7_my = planes[(planes["seats"].between(150, 210)) & (planes["year"] >= 2011)][["tailnum"]]
task7_my = task7_my.sort_values("tailnum").reset_index(drop=True)

assert_frame_equal(task7_sql, task7_my)

This query filters the planes dataset to only include planes that have between 150 and 210 seats and were manufactured in 2011 or later. From the filtered planes, it returns only their tail numbers.

```sql
SELECT tailnum, manufacturer, seats FROM planes
WHERE manufacturer IN ("BOEING", "AIRBUS", "EMBRAER") AND seats>390
```

In [43]:
task8_sql = pd.read_sql_query("""
SELECT tailnum, manufacturer, seats FROM planes
WHERE manufacturer IN ("BOEING", "AIRBUS", "EMBRAER") AND seats>390 ORDER BY tailnum, manufacturer, seats
""", conn)

task8_my = planes[planes["manufacturer"].isin(["BOEING", "AIRBUS", "EMBRAER"]) & (planes["seats"] > 390)][
    ["tailnum", "manufacturer", "seats"]]
task8_my = task8_my.sort_values(["tailnum", "manufacturer", "seats"]).reset_index(drop=True)

assert_frame_equal(task8_sql, task8_my)

This query selects planes made by Boeing, Airbus, or Embraer that have more than 390 seats. From these filtered planes, it returns only the tail number, manufacturer, and seat count.

```sql
SELECT DISTINCT year, seats FROM planes
WHERE year >= 2012 ORDER BY year ASC, seats DESC
```

In [44]:
task9_sql = pd.read_sql_query("""
SELECT DISTINCT year, seats FROM planes
WHERE year >= 2012 ORDER BY year ASC, seats DESC
""", conn)

task9_my = (
    planes[planes["year"] >= 2012][["year", "seats"]]
    .drop_duplicates()
    .sort_values(["year", "seats"], ascending=[True, False])
    .reset_index(drop=True)
)

assert_frame_equal(task9_sql, task9_my)

This query filters the planes table to include only planes manufactured in 2012 or later.
It then selects only the year and seats columns, removes duplicate combinations, and finally sorts the result so that newer years appear later and, within each year, planes with more seats appear first.

```sql
SELECT DISTINCT year, seats FROM planes
WHERE year >= 2012 ORDER BY seats DESC, year ASC
```

In [45]:
task10_sql = pd.read_sql_query("""
SELECT DISTINCT year, seats FROM planes
WHERE year >= 2012 ORDER BY seats DESC, year ASC
""", conn)

task10_my = (
    planes[planes["year"] >= 2012][["year", "seats"]]
    .drop_duplicates()
    .sort_values(["seats", "year"], ascending=[False, True])
    .reset_index(drop=True)
)
assert_frame_equal(task10_sql, task10_my)

This query filters the planes dataset to rows where the year is 2012 or later.
It then keeps the unique (year, seats) combinations and sorts them so that planes with more seats appear first (descending seats), and for ties in seat count, earlier manufacturing years come first (ascending year).

```sql
SELECT manufacturer, COUNT(*) FROM planes
WHERE seats > 200 GROUP BY manufacturer
```

In [46]:
task11_sql = pd.read_sql_query("""
SELECT manufacturer, COUNT(*) FROM planes
WHERE seats > 200 GROUP BY manufacturer
""", conn).sort_values("manufacturer").reset_index(drop=True)

task11_my = (
    planes[planes["seats"] > 200]
    .groupby("manufacturer")
    .size()
    .reset_index(name="COUNT(*)")
).sort_values("manufacturer").reset_index(drop=True)
assert_frame_equal(task11_sql, task11_my)

This query filters the planes table to include only aircraft with more than 200 seats.
Then it groups the remaining rows by manufacturer and counts how many planes each manufacturer has in that category.

```sql
SELECT manufacturer, COUNT(*) FROM planes
GROUP BY manufacturer HAVING COUNT(*) > 10
```

In [47]:
task12_sql = pd.read_sql_query("""
SELECT manufacturer, COUNT(*) FROM planes
GROUP BY manufacturer HAVING COUNT(*) > 10
""", conn).sort_values("manufacturer").reset_index(drop=True)

task12_my = (
    planes
    .groupby("manufacturer")
    .size()
    .reset_index(name="COUNT(*)")
)
task12_my = task12_my[task12_my["COUNT(*)"] > 10].sort_values("manufacturer").reset_index(drop=True)

assert_frame_equal(task12_sql, task12_my)

This query groups the planes table by manufacturer, counts how many planes each manufacturer has, and then keeps only the manufacturers that appear more than ten times. It returns each qualifying manufacturer along with the corresponding aircraft count.

```sql
SELECT manufacturer, COUNT(*) FROM planes
WHERE seats > 200 GROUP BY manufacturer HAVING COUNT(*) > 10
```

In [48]:
task13_sql = pd.read_sql_query("""
SELECT manufacturer, COUNT(*) FROM planes
WHERE seats > 200 GROUP BY manufacturer HAVING COUNT(*) > 10
""", conn).sort_values("manufacturer").reset_index(drop=True)

task13_my = (
    planes[planes["seats"] > 200]
    .groupby("manufacturer")
    .size()
    .reset_index(name="COUNT(*)")
)

task13_my = task13_my[task13_my["COUNT(*)"] > 10].sort_values("manufacturer").reset_index(drop=True)

assert_frame_equal(task13_sql, task13_my)

This query first filters the planes dataset to only include aircraft with more than 200 seats.
It then groups the remaining rows by manufacturer and counts the number of planes in each group.
Finally, it returns only those manufacturers that have more than ten planes in the filtered dataset.

```sql
SELECT manufacturer, COUNT(*) AS howmany
FROM planes
GROUP BY manufacturer
ORDER BY howmany DESC LIMIT 10
```

In [49]:
task14_sql = pd.read_sql_query("""
SELECT manufacturer, COUNT(*) AS howmany
FROM planes
GROUP BY manufacturer
ORDER BY howmany DESC LIMIT 10
""", conn)

task14_my = (
    planes
    .groupby("manufacturer")
    .size()
    .reset_index(name="howmany")
    .sort_values("howmany", ascending=False)
    .head(10)
    .reset_index(drop=True)
)

assert_frame_equal(task14_sql, task14_my)

This query groups the planes table by manufacturer and counts how many planes each manufacturer has in the dataset.
It then sorts the manufacturers by that count in descending order and returns only the top 10 manufacturers with the most planes.

```sql
SELECT
flights.*,
planes.year AS plane_year,
planes.speed AS plane_speed,
planes.seats AS plane_seats
FROM flights LEFT JOIN planes ON flights.tailnum=planes.tailnum
```

In [50]:
task15_sql = pd.read_sql_query("""
SELECT
flights.*,
planes.year AS plane_year,
planes.speed AS plane_speed,
planes.seats AS plane_seats
FROM flights LEFT JOIN planes ON flights.tailnum=planes.tailnum
""", conn)

planes_subset = planes[["tailnum", "year", "speed", "seats"]]
# Rename the new columns to match SQL
planes_subset = planes_subset.rename(columns={
    "year": "plane_year",
    "speed": "plane_speed",
    "seats": "plane_seats"
})

# LEFT JOIN flights by planes_subset
task15_my = flights.merge(
    planes_subset,
    on="tailnum",
    how="left"
)
# Convert pandas NaN to None
task15_my = task15_my.where(pd.notnull(task15_my), None)

common_cols = flights.columns.tolist()
task15_sql = task15_sql.sort_values(common_cols).reset_index(drop=True)
task15_my = task15_my.sort_values(common_cols).reset_index(drop=True)

assert_frame_equal(task15_sql, task15_my)

The SQL query performs a LEFT JOIN of the flights table with the planes table on the tailnum field.
For each flight, it adds plane information (year, speed, seats).
If a flight has no matching airplane record, the new fields are NULL.

The pandas version uses merge(..., how="left") to achieve the same effect, then renames the columns to match the SQL output.

```sql
SELECT planes.*, airlines.* FROM
(SELECT DISTINCT carrier, tailnum FROM flights) AS cartail
INNER JOIN planes ON cartail.tailnum=planes.tailnum
INNER JOIN airlines ON cartail.carrier=airlines.carrier
```

In [51]:
task16_sql = pd.read_sql_query("""
SELECT planes.*, airlines.*
FROM
    (SELECT DISTINCT carrier, tailnum FROM flights) AS cartail
    INNER JOIN planes   ON cartail.tailnum = planes.tailnum
    INNER JOIN airlines ON cartail.carrier = airlines.carrier
""", conn)

distinct_cartail = flights[["carrier", "tailnum"]].drop_duplicates()
inner_join_planes = distinct_cartail.merge(planes, on="tailnum", how="inner")
task16_my = inner_join_planes.merge(airlines, on="carrier", how="inner")

# Reorder columns: planes.*, airlines.*
planes_cols = planes.columns.tolist()
airlines_cols = airlines.columns.tolist()
task16_my = task16_my[planes_cols + airlines_cols]

# Compare (normalize NULL vs NaN)
sort_cols = planes_cols + airlines_cols
task16_sql = task16_sql.sort_values(sort_cols).reset_index(drop=True)
task16_my = task16_my.sort_values(sort_cols).reset_index(drop=True)

assert_frame_equal(task16_sql, task16_my)

The SQL query first extracts all unique (carrier, tailnum) pairs from the flights dataset.
Then it joins this set with the planes table using tailnum and joins with the airlines table using carrier.
The result contains all matching rows from both joins, and returns all columns from planes and all columns from airlines.

```sql
SELECT
flights2.*,
atemp,
ahumid
FROM (
SELECT * FROM flights WHERE origin='EWR'
) AS flights2
LEFT JOIN (
SELECT
year, month, day,
AVG(temp) AS atemp,
AVG(humid) AS ahumid
FROM weather
WHERE origin='EWR'
GROUP BY year, month, day
) AS weather2
ON flights2.year=weather2.year
AND flights2.month=weather2.month
AND flights2.day=weather2.day
```

In [52]:
task17_sql = pd.read_sql_query("""
SELECT
flights2.*,
atemp,
ahumid
FROM (
SELECT * FROM flights WHERE origin='EWR'
) AS flights2
LEFT JOIN (
SELECT
year, month, day,
AVG(temp) AS atemp,
AVG(humid) AS ahumid
FROM weather
WHERE origin='EWR'
GROUP BY year, month, day
) AS weather2
ON flights2.year=weather2.year
AND flights2.month=weather2.month
AND flights2.day=weather2.day
""", conn)

flights2 = flights[flights["origin"] == "EWR"]
weather2 = (
    weather[weather["origin"] == "EWR"]
    .groupby(["year", "month", "day"], as_index=False)
    .agg(
        atemp=("temp", "mean"),
        ahumid=("humid", "mean")
    )
)

task17_my = flights2.merge(
    weather2,
    on=["year", "month", "day"],
    how="left"
)

common_cols = flights2.columns.tolist()

task17_sql = task17_sql.sort_values(common_cols).reset_index(drop=True)
task17_my  = task17_my.sort_values(common_cols).reset_index(drop=True)
task17_my = task17_sql.where(pd.notnull(task17_my), None)

assert_frame_equal(task17_sql, task17_my)

First, we filter the flights table to only include flights whose origin is EWR (Newark).
Then, we take the weather table, keep only rows from EWR, and group them by date (year, month, day) to compute the average temperature (atemp) and average humidity (ahumid) for each day.
Finally, we perform a left join of the EWR flights with this aggregated weather table on the date, so each EWR flight gets the corresponding daily average temperature and humidity attached.
Flights on days without weather records get NULL/NaN in atemp and ahumid.

## Conclusion
Overall, `pandas` is a powerful and versatile Python library that enables efficient filtering, manipulation, and transformation of datasets. Through the examples presented in this report, readers can gain a solid understanding of how `pandas` can replicate and extend many SQL-like operations. By mastering these techniques, users can take full advantage of `pandas` for data analysis in a flexible, Python-based workflow.

## Reference
[pandas](https://pandas.pydata.org/docs/reference/index.html)
[sqlite3](https://docs.python.org/3/library/sqlite3.html)